<a href="https://colab.research.google.com/github/SohomCh/ANN_Customer_Churn/blob/main/Customer_churn_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
import numpy as np
import pandas as pd
import pickle
from tensorflow.keras.models import load_model

In [35]:
# 1. Load model
model = load_model("model.keras")

# 2. Load encoders & scaler
with open("label_encoder_gender", "rb") as f:
    label_encoder_gender = pickle.load(f)

with open('onehot_encoder_geo.pkl','rb') as file:
    label_encoder_geo = pickle.load(file)

with open("scaler.pkl", "rb") as f:
    scaler = pickle.load(f)

/usr/local/lib/python3.12/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator OneHotEncoder from version 1.5.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.5.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [36]:
input_data = {
    "CreditScore": 600,
    "Geography": "France",
    "Gender": "Male",
    "Age": 40,
    "Tenure": 3,
    "Balance": 60000,
    "NumOfProducts": 2,
    "HasCrCard": 1,
    "IsActiveMember": 1,
    "EstimatedSalary": 50000
}


In [37]:
# In [7]:
# One-hot encode 'Geography'
geo_encoded = label_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(
    geo_encoded,
    columns=label_encoder_geo.get_feature_names_out(['Geography'])
)
geo_encoded_df


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [38]:
# In [9]:
input_df = pd.DataFrame([input_data])
input_df


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [39]:
# In [10]:
## Encode categorical variables
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])
input_df


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [40]:
# In [11]:
## Concatenate one-hot encoded
input_df = pd.concat([input_df.drop("Geography", axis=1), geo_encoded_df], axis=1)
input_df


,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [43]:
feature_order = [
    "CreditScore", "Gender", "Age", "Tenure", "Balance",
    "NumOfProducts", "HasCrCard", "IsActiveMember",
    "EstimatedSalary", "Geography_France", "Geography_Germany", "Geography_Spain"
]

# Encode features
gender_encoded = label_encoder_gender.transform([input_data["Gender"]])[0]
geo_encoded = label_encoder_geo.transform([[input_data["Geography"]]]).toarray()[0]

row_values = [
    input_data["CreditScore"],
    gender_encoded,
    input_data["Age"],
    input_data["Tenure"],
    input_data["Balance"],
    input_data["NumOfProducts"],
    input_data["HasCrCard"],
    input_data["IsActiveMember"],
    input_data["EstimatedSalary"],
    *geo_encoded
]

print("Length of row_values:", len(row_values))  # should be 12

# Create DataFrame
input_df = pd.DataFrame([row_values], columns=feature_order)


Length of row_values: 12


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [44]:
input_scaled = scaler.transform(input_df)


In [45]:
prediction = model.predict(input_scaled)
print("Churn probability:", prediction[0][0])


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 166ms/step
Churn probability: 0.22268057


In [46]:
if prediction[0][0] >= 0.5:
    print("Prediction: Customer will churn")
else:
    print("Prediction: Customer will NOT churn")


Prediction: Customer will NOT churn
